In [1]:
import wikipedia
import wikipediaapi 
import datetime
import pandas as pd
import numpy as np
from sklearn import feature_extraction
from sklearn import model_selection
import nltk
from nltk import tokenize
from nltk import word_tokenize, pos_tag
import string
from keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import multiclass
from sklearn import metrics
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
classes = ['sport', 'economy', 'politics', 'health', 'tourism', 
           'entertainment', 'culture', 'music', 'education', 
           'real estate', 'nutrition', 'science', 'history', 'technology'
          ]

In [3]:
def generate_data(classes, n):
    data = [];
    for i in classes:
#         topics = wikipedia.search(i, results = n)
#         for j in topics:
#         print(classes)
#         print("Class: ", i)
        for j in list(wikipedia.search(i,results = n)):
            wiki_wiki = wikipediaapi.Wikipedia('en')
            page_py = wiki_wiki.page(j)
            if(page_py.exists()):
#                 print("Class: ", i)
#                 print("Title: ", j)
#                     print("Section: ", page_py.sections[0].title)
#                     print("Text: ", page_py.text)
                for k in page_py.categories:
                    if i in k.lower(): 
#                         print("Class: ", i)
                        data.append((i, page_py.summary))
                        break
            else:
                continue
    return data

In [5]:
data = generate_data(classes, 10)

In [6]:
for i in classes:
    c = 0
    for j in range(0, len(data)):
        if(data[j][0] == i):
            c = c + 1
    print("%s %d" %(i, c))

sport 10
economy 4
politics 6
health 7
tourism 9
entertainment 5
culture 9
music 7
education 9
real estate 9
nutrition 8
science 8
history 6
technology 8


In [7]:
def create_data_csv_file(name, data):
    name = name + '.csv'
#         '_' + str(datetime.datetime.now()).replace(' ', '_').replace('.', '-').replace(':', '-') + 
#         '.csv'
    sep = '<>'
    cols = 'class' + sep + 'text\n'
    with open(name, 'w+', encoding='utf-8') as file:
        file.write(cols)
#         YELLOW,SMALL,STRETCH,ADULT,T
        for i in range(0, len(data)):
            line = data[i][0] + sep + data[i][1].replace('\n', ' ')

            file.write(line)
            file.write('\n')
    file.close()
            
#     print(text)

In [9]:
# create_data_csv_file('data_10', data)

In [10]:
df = pd.read_csv('data_10.csv', sep = '<>')
df

<ipython-input-10-79470950938f>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('data_10.csv', sep = '<>')


,class,text
0,sport,Sport pertains to any form of competitive phys...
1,sport,Sport was an American sports magazine. Launche...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп..."
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...
4,sport,Sport diving is an underwater sport that uses ...
...,...,...
100,technology,"Fintech a portmanteau of ""financial technology..."
101,technology,TCL Technology (originally an abbreviation for...
102,technology,Space technology is technology for use in oute...
103,technology,A technology company (or tech company) is an e...


In [11]:
df['class'].value_counts()

sport            10
education         9
tourism           9
real estate       9
culture           9
technology        8
nutrition         8
science           8
music             7
health            7
politics          6
history           6
entertainment     5
economy           4
Name: class, dtype: int64

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Palja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
dict_class_index = {
    class_name: i for i, class_name in enumerate(df['class'].unique())
}

df['class num'] = df['class'].map(dict_class_index)

In [14]:
df

,class,text,class num
0,sport,Sport pertains to any form of competitive phys...,0
1,sport,Sport was an American sports magazine. Launche...,0
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0
4,sport,Sport diving is an underwater sport that uses ...,0
...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13
101,technology,TCL Technology (originally an abbreviation for...,13
102,technology,Space technology is technology for use in oute...,13
103,technology,A technology company (or tech company) is an e...,13


In [15]:
dict_class_index

{'sport': 0,
 'economy': 1,
 'politics': 2,
 'health': 3,
 'tourism': 4,
 'entertainment': 5,
 'culture': 6,
 'music': 7,
 'education': 8,
 'real estate': 9,
 'nutrition': 10,
 'science': 11,
 'history': 12,
 'technology': 13}

In [20]:
import advertools as adv
stop_words = sorted(adv.stopwords['english'])
print(stop_words)

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon

In [21]:
def clear_text(text):
    new_text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])).str.lower()
    new_text = new_text.replace(r'[^\w\s]+', '', regex=True)
    return new_text

In [22]:
new_text = clear_text(df['text'])

In [24]:
df['clear text'] = new_text
df

,class,text,class num,clear text
0,sport,Sport pertains to any form of competitive phys...,0,sport pertains form competitive physical activ...
1,sport,Sport was an American sports magazine. Launche...,0,sport american sports magazine launched septem...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0,sport sport sport russian спорт спорт спорт 19...
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0,sport lisboa e benfica portuguese pronunciatio...
4,sport,Sport diving is an underwater sport that uses ...,0,sport diving underwater sport uses recreationa...
...,...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13,fintech portmanteau financial technology refer...
101,technology,TCL Technology (originally an abbreviation for...,13,tcl technology originally abbreviation telepho...
102,technology,Space technology is technology for use in oute...,13,space technology technology use outer space tr...
103,technology,A technology company (or tech company) is an e...,13,a technology company or tech company electroni...
